In [177]:
# compilation des tests pour simplification
from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.transforms as transforms
import cv2
import glob
from os.path import basename
import tools as to
import fishermen as fi
from torch.autograd import Variable
from importlib import reload
import sys
reload(fi)

<module 'fishermen' from 'C:\\Users\\dambr\\obelix\\projets\\pytorch\\fishermen.py'>

In [25]:
0# Chose your input parameters 
root_folder="C:/Users/dambr/wipsea/projets/cifre_mathieu/experience/pecheurs/"
out_folder = root_folder + "patches/"
posRatio = 0.5
patch_width = 32
patch_height = 64
print("root_folder is : \n" + root_folder)
print("Patches will be stored in output folder : \n" + out_folder)
print("Positive patch ratio per image : " + str(posRatio))
print("Patch size is : width = %d, height = %d" % (patch_width, patch_height))

root_folder is : 
C:/Users/dambr/wipsea/projets/cifre_mathieu/experience/pecheurs/
Patches will be stored in output folder : 
C:/Users/dambr/wipsea/projets/cifre_mathieu/experience/pecheurs/patches/
Positive patch ratio per image : 0.5
Patch size is : width = 32, height = 64


In [121]:
fisherman_dataset = fi.FishermanDataset(csv_file=out_folder +'Annotations.csv',
                                    root_dir=out_folder, 
                                       transform=transforms.Compose([
                                               fi.Rescale((70,35)),
                                               fi.RandomCrop((64,32)),
                                               fi.ToTensor()
                                           ]))

mean, std = fi.calcNorm(fisherman_dataset)
print(mean)
print(std)
fisherman_norm = fi.FishermanDataset(csv_file=out_folder +'Annotations.csv',
                                    root_dir=out_folder, 
                                       transform=transforms.Compose([
                                               fi.Rescale((70,35)),
                                               fi.RandomCrop((64,32)),
                                               fi.ToTensor(),
                                               fi.Normalize(tuple(mean),tuple(std))
                                           ]))

mean, std = fi.calcNorm(fisherman_norm)
print(mean)
print(std)

[0.51619951 0.52364265 0.53038227]
[0.05715425 0.05635047 0.05636032]
[-0.0007836   0.00055154  0.00176748]
[0.99776958 0.99758771 0.99762201]


In [203]:
trainset_unnormalized = fi.FishermanDataset(csv_file=out_folder + "Annotations.csv",
                                            root_dir=out_folder,
                                            transform = fi.ToTensor()
                                         )
mean, std = fi.calcNorm(trainset_unnormalized)
print(mean)
print(std)
composed_train = transforms.Compose([fi.Rescale((70,35)),
                                     fi.RandomCrop((64,32)),
                                     fi.ToTensor(),
                                     fi.Normalize((0.5,0.5,0.5),(1,1,1))
                                     ])

composed_test = transforms.Compose(
    [fi.ToTensor(),
     fi.Normalize((0.5,0.5,0.5),(1,1,1))])

trainset = fi.FishermanDataset(csv_file=out_folder + "Annotations.csv",
                                      root_dir=out_folder,
                                      transform = composed_train)

trainloader = DataLoader(trainset, batch_size=4,
                         shuffle=True)

testset = fi.FishermanDataset(csv_file=out_folder + "Annotations_test.csv",
                                      root_dir=out_folder,
                                      transform = composed_test)
s = testset[2]
im = s['image']
print(im[2,20,20])

testloader = DataLoader(testset, batch_size=1,
                         shuffle=False)

[0.51706663 0.52491365 0.53139331]
[0.05846014 0.05763357 0.05749327]
tensor(-0.1353, dtype=torch.float64)


In [126]:
#3 Create a network
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, (3,3))
        self.pool = nn.MaxPool2d(kernel_size=(2,2))
        self.conv2 = nn.Conv2d(5, 16, (3,3))
        self.line3 = nn.Linear(16 * 6 * 14, 20)
        self.line4 = nn.Linear(20, 10)
        self.line5 = nn.Linear(10, 1)
        
    def forward(self, x):
        #print(x.size())
        x = self.pool(F.relu(self.conv1(x)))
        #print(x.size())
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.size())
        x = x.view(-1, 16 * 6 * 14)
        #print(x.size())
        x = F.relu(self.line3(F.dropout(x,0.1,False,True)))
        #print(x.size())
        x = F.relu(self.line4(F.dropout(x,0.1,False,True)))
        #print(x.size())
        x = self.line5(x)
        #print(x.size())
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
params = list(net.parameters())
print("params number")
print(to.get_n_params(net))


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

params number
28429


Net(
  (conv1): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(5, 16, kernel_size=(3, 3), stride=(1, 1))
  (line3): Linear(in_features=1344, out_features=20, bias=True)
  (line4): Linear(in_features=20, out_features=10, bias=True)
  (line5): Linear(in_features=10, out_features=1, bias=True)
)

In [204]:
#4 define metaparameter
import torch.optim as optim

#criterion = nn.CrossEntropyLoss()
criterion = nn.SoftMarginLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [205]:
e = 10
show = 20
to.trainMyNet(net,trainloader,optimizer,criterion,e,show,device)

[1,    20] loss: 1.395
[1,    40] loss: 0.696
[1,    60] loss: 0.695
[1,    80] loss: 0.696
[1,   100] loss: 0.692
[1,   120] loss: 0.703
[1,   140] loss: 0.676
[1,   160] loss: 0.715
[1,   180] loss: 0.698
[1,   200] loss: 0.703
[2,    20] loss: 0.696
[2,    40] loss: 0.698
[2,    60] loss: 0.696
[2,    80] loss: 0.690
[2,   100] loss: 0.697
[2,   120] loss: 0.692
[2,   140] loss: 0.704
[2,   160] loss: 0.694
[2,   180] loss: 0.695
[2,   200] loss: 0.690
[3,    20] loss: 0.695
[3,    40] loss: 0.696
[3,    60] loss: 0.695
[3,    80] loss: 0.684
[3,   100] loss: 0.724
[3,   120] loss: 0.694
[3,   140] loss: 0.696
[3,   160] loss: 0.694
[3,   180] loss: 0.694
[3,   200] loss: 0.693
[4,    20] loss: 0.693
[4,    40] loss: 0.691
[4,    60] loss: 0.692
[4,    80] loss: 0.694
[4,   100] loss: 0.693
[4,   120] loss: 0.697
[4,   140] loss: 0.698
[4,   160] loss: 0.689
[4,   180] loss: 0.699
[4,   200] loss: 0.697
[5,    20] loss: 0.695
[5,    40] loss: 0.696
[5,    60] loss: 0.694
[5,    80] 

In [209]:
accuracy = to.testMyNet(net,testloader, device)

Accuracy of the network on the 93 test images: 94 %


In [207]:
#7 Save my net
is_best = True
to.save_checkpoint({
    'epoch': e + 1,
    'arch': 'GPU',
    'state_dict': net.state_dict(),
    'best_prec1': accuracy,
    'optimizer' : optimizer.state_dict(),
    }, is_best)

In [208]:
model = Net()
checkpoint = torch.load('C:/Users/dambr/obelix/projets/pytorch/model_best.pth.tar')
e = checkpoint['epoch']
best_prec1 = checkpoint['best_prec1']
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.to(device)

Net(
  (conv1): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(5, 16, kernel_size=(3, 3), stride=(1, 1))
  (line3): Linear(in_features=1344, out_features=20, bias=True)
  (line4): Linear(in_features=20, out_features=10, bias=True)
  (line5): Linear(in_features=10, out_features=1, bias=True)
)

In [266]:
reload(fi)
test_features = np.ones((1,11))
layer = model._modules.get('line4')
with torch.no_grad():
    for data in testloader:
        feat = fi.get_vectors(data, model, layer, device)
        f = feat.data[0].numpy()
        line = np.zeros((1,11))
        line[0,:-1] = f
        line[0,10] = float(data['label'].data[0])
        test_features = np.append(test_features,line, axis=0)
        #test_features.append(line)
print("Res")
print(test_features[50])
df = pd.DataFrame({'f1' : test_features[:,0],
                   'f2' : test_features[:,1],
                   'f3' : test_features[:,2],
                   'f4' : test_features[:,3],
                   'f5' : test_features[:,4],
                   'f6' : test_features[:,5],
                   'f7' : test_features[:,6],
                   'f8' : test_features[:,7],
                   'f9' : test_features[:,8],
                   'f10' : test_features[:,9],
                   'label' : test_features[:,10]})
df.to_csv("test_write.csv", ',')

Res
[ -3.33665633   6.76845598   0.28124112  -3.1149168   -2.33480549
  -0.4990955  -12.7186985   -2.47497559   7.57247305  -1.7359612
   1.        ]
